In [1]:
# Installing ERNIE
%tensorflow_version 2.x
!pip install ernie==0.0.33b0 > /dev/null

In [2]:
#Importing libraries
from sklearn.metrics import accuracy_score
from ernie import SentenceClassifier, Models
import pandas as pd

In [3]:
#Getting data in dataframe
df = pd.read_csv("Harassment_Cleaned_tweets.csv")
df.head()

,Unnamed: 0,Key Word,Username,User_ID,Datetime,Favorite_count,Geo,Coordinates,Label,Text,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,704,ass,DeborahParr,1.330000e+18,11-11-2020 06:56,0,NaN,NaN,1,He’d have my phone wedged up his ass sideways.,NaN,NaN,NaN
1,1915,boobies,MaxZorin85,1.330000e+18,11-11-2020 07:35,4,NaN,NaN,0,Yep 100% agree and the same with severine in s...,NaN,NaN,NaN
2,2856,eat pussy,PRISJ1_,1.330000e+18,11-11-2020 10:36,0,NaN,NaN,1,Stop having sex with men that won’t eat your p...,NaN,NaN,NaN
3,2163,Breast Man,Teresamckenzy1,1.330000e+18,10-11-2020 20:52,0,NaN,NaN,1,"When you see a sad man, just give him breast t...",NaN,NaN,NaN
4,2852,eat pussy,sj__vazquez,1.330000e+18,11-11-2020 10:42,0,NaN,NaN,1,We can't be together if you don't eat pussy,NaN,NaN,NaN


In [4]:
#Train Test split 0.8-0.2
train = df[:2883]
test = df[2883:]

In [5]:
#Converting Train df into list of tuples
listt = []

for i in range (0,len(train)):
  listt.append((train.at[i,"Text"],train.at[i,"Label"]))


In [6]:
df2 = pd.DataFrame(listt)

In [7]:
#Roberta Base Cased Ernie
classifier = SentenceClassifier(model_name=Models.RobertaBaseCased, max_length=64)
classifier.load_dataset(df2, validation_split=0.1)
classifier.fine_tune(epochs=5, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2021-06-04 09:12:58 [WARNING] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/array_ops.py:5049: calling gather (from tensorflow.python.ops.array_ops) with validate_indices is deprecated and will be removed in a future version.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2021-06-04 09:13:04 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2021-06-04 09:13:14 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


81/81 [==============================] - 70s 857ms/step - loss: 0.1536 - accuracy: 0.9498 - val_loss: 0.5150 - val_accuracy: 0.8242


In [8]:
#Test on custom input
sentence = "Show them them titties"
probabilities = classifier.predict_one(sentence)
print(type(probabilities[0]))

<class 'float'>


In [9]:
#Get y_preds list
y_pred = []

for i in range (2883,len(df)):
  probabilities = classifier.predict_one(test.at[i,"Text"])
  if probabilities[0]>probabilities[1]:
    y_pred.append(0)
  else:
    y_pred.append(1)

In [10]:
y_true = test["Label"].tolist()
print (accuracy_score(y_true, y_pred))

0.8210818307905686


In [11]:
#BERT Base Cased Ernie
classifier = SentenceClassifier(model_name=Models.BertBaseCased, max_length=64)
classifier.load_dataset(df2, validation_split=0.1)
classifier.fine_tune(epochs=5, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)


81/81 [==============================] - 69s 843ms/step - loss: 0.0940 - accuracy: 0.9688 - val_loss: 0.7338 - val_accuracy: 0.8008


In [12]:
#Get y_preds list
y_pred = []

for i in range (2883,len(df)):
  probabilities = classifier.predict_one(test.at[i,"Text"])
  if probabilities[0]>probabilities[1]:
    y_pred.append(0)
  else:
    y_pred.append(1)

In [13]:
y_true = test["Label"].tolist()
print (accuracy_score(y_true, y_pred))

0.8058252427184466
